# 02 – Using a real ML framework in Docker

This example shows how Dockyter can run a **full machine learning framework** from a notebook without installing it into the base Python environment.

Instead of doing `pip install torch` (or tensorflow, or any other heavy stack) in the notebook kernel, we use a Docker image that already contains everything we need. The notebook only needs:

- a normal Python kernel, and
- the `dockyter` IPython extension.

All the heavy dependencies (Pytorch, TensorFlow, CUDA/BLAS libraries, etc.) stay inside the Docker image, which can be reused across different projects and environments.


In [1]:
# Load the Dockyter IPython extension
%load_ext dockyter

## Example: run a small PyTorch training loop inside Docker

In the cell below, we use the official `pytorch/pytorch` image and run a tiny training loop entirely inside the container.

From the notebook’s point of view:

- there is **no PyTorch installed in the base environment**,  
- all imports (`import torch`, etc.) happen inside Docker,  
- Dockyter wires `%%docker` to `docker run ...` so the notebook can still drive the experiment.

The example creates a fake dataset, builds a small `nn.Sequential` model, and trains it for a few epochs, printing the loss at each step.

In [2]:
%%docker pytorch/pytorch
python - << 'EOF'
import torch
import torch.nn as nn
import torch.optim as optim

print("PyTorch version:", torch.__version__)

# Fake dataset
x = torch.randn(1000, 10)
y = (x.sum(dim=1) > 0).float().unsqueeze(1)

model = nn.Sequential(
    nn.Linear(10, 16),
    nn.ReLU(),
    nn.Linear(16, 1),
    nn.Sigmoid(),
)

criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

print("Starting training inside Docker...")
for epoch in range(3):
    optimizer.zero_grad()
    y_pred = model(x)
    loss = criterion(y_pred, y)
    loss.backward()
    optimizer.step()
    print(f"Epoch {epoch+1}, loss={loss.item():.4f}")
EOF


PyTorch version: 2.2.1
Starting training inside Docker...
Epoch 1, loss=0.7421
Epoch 2, loss=0.7398
Epoch 3, loss=0.7376

